# rohe beschreibung was in den codes gemacht wird und wieso

oft wird rag gebaut mit pdfs, und wenn man so ein system aufbaut kann man sicherlich eine grössere user base ansprechen, welche das system dann auch bauen möchten. in diesem projekt möchte ich aber webscraping von yahoo finance machen, weil viele leute auch yahoo finance lesen und auf yahoo finance viel mehr informationen sind. ich kann dort webscraping machen von bestimmten aktien, fonds oder indexes und dann alle informationen scrapen von kursen, artikeln, stock prices,
intraday trading data,
option prices;
company disclosures and information such as :
financial information,
earnings,
analyst ratings and recommendations,
insider trades,
SEC filings,
news,
large shareholders;

 dann kann ich all das in einen ordner speichern und einen rag bauen auf all diese sachen. das spart einerseits zeit weil man nicht manuell nach pdfs online suchen muss zu den gewünschten themen, sondern man kann alles zeitsparend an einem ort scrapen und dann weiterbenutzen als input für das rag.

 denn technisch gesehen macht LangChain keinen Unterschied, ob deine Daten ursprünglich aus PDFs, TXTs, Webseiten oder JSON stammen. Wichtig ist nur, was in den vektorstore eingespeist wird, ist reiner text, nicht das ursprüngliche Dateiformat (wie z.b. pdf).

 Ob du folgenden Text aus einem PDF extrahierst:
 In 2023, Allianz reported a revenue of €152.7 billion and a net income of €14.2 billion.

oder denselben Text aus einem JSON-Feld zusammenstellst wie:
{ "revenue": "€152.7 billion", "net_income": "€14.2 billion" }

ist für den Vektorstore und LangChain völlig egal, solange du am Ende diesen Text als Chunk übergibst.

📘 Warum sieht man online meist PDFs?

PDFs sind realistische Quellen (z. B. Geschäftsberichte, Studien)
Sie demonstrieren komplexere Verarbeitung (PDF → Text → Chunk → Embedding)
Viele Nutzer haben genau solche Dateien → Tutorials wollen für die breite Masse relevant sein
JSON ist oft maschinenlesbar, aber in der Praxis weniger "typisch" für Endnutzer
✅ Wann ist JSON sogar besser?

Wenn du strukturierte, tabellarische Daten hast (Finanzdaten, ESG-KPIs, Zeitreihen)
Wenn du Daten aus APIs sammelst (z. B. von Morningstar, AlphaVantage, Unternehmensdatenbanken)
Wenn du dein System voll automatisiert aufbaust ohne manuelles PDF-Verarbeiten
Wenn du bestimmte Felder gezielt extrahieren und in Textform bringen willst

Ja, du kannst problemlos ein LangChain-basiertes RAG-System mit JSON-Dateien aufbauen.
🧱 Die einzige Voraussetzung: Du musst den Inhalt in Klartext-Chunks bringen, damit das LLM damit arbeiten kann.

für jede firma und für jeden datensatz (aktienkurs, cashflow etc.) wird jeweils ein json datei generiert aus folgendem grund:

1. Modular & skalierbar

Wenn jede Bank ihr eigenes Verzeichnis bzw. JSON-Set hat, kannst du:

gezielt einzelne Firmen updaten (nur JPM aktualisieren statt alles neu scrapen)
schneller auf firmenspezifische Daten zugreifen
einfacher firmenspezifische Claims prüfen (z. B. „Wie war der Cashflow von JPM?“)

2. Sauberer für Chunking & RAG

Ein RAG-System arbeitet oft dokumentenbasiert. Wenn jede Firma ein „Dokument“ bzw. Datensatz ist, kannst du:

firmenspezifische Chunks erzeugen
Kontext klar steuern: "Context: JPMorgan" → keine Gefahr durch „vermischte“ Daten
Retrieval filtern nach company_name, ticker, etc.

3. Einfachere Annotation / Fact-Checking

Für deinen Fragebogen willst du Claims wie:

„JPMorgan hatte 2024 einen positiven Free Cash Flow.“
→ Da ist es superpraktisch, wenn du direkt auf JPM/cashflow.json zugreifen kannst – statt alles aus einem Riesendokument zu parsen.

# source .venv/bin/activate 

In [23]:
import yfinance as yf
import pandas as pd
import matplotlib
from datetime import datetime
from pytz import timezone
import os
import json


# Masterfile, das alles webscraped mit yfinance und in einzelnen json files pro unternehmen abspeichert


In [ ]:
# ✅ Tickerliste: einfach erweiterbar
tickers = ['JPM', 'BAC', 'C', 'UBSG.SW', 'HSBA.L', 'BNP.PA']

# 📁 Basisordner
base_path = "data"

# 🔁 Hilfsfunktion zum Speichern von JSON (inkl. datetime-fix)
def save_json(data, path):
    def json_converter(o):
        if isinstance(o, (datetime.datetime, datetime.date)):
            return o.isoformat()
        return str(o)  # Fallback für andere exotische Typen

    try:
        if isinstance(data, pd.DataFrame):
            if data.empty:
                print(f"⚠️ Leere DataFrame: {path}")
                return
            data = data.reset_index()
            data.columns = [str(col) for col in data.columns]
            for col in data.columns:
                if pd.api.types.is_datetime64_any_dtype(data[col]):
                    data[col] = data[col].dt.strftime('%Y-%m-%d')
            data = data.to_dict(orient="records")
        elif isinstance(data, (dict, list)):
            pass
        else:
            print(f"⚠️ Nicht serialisierbar: {path}")
            return

        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False, default=json_converter)

        print(f"✅ Gespeichert: {path}")
    except Exception as e:
        print(f"❌ Fehler beim Speichern von {path}: {e}")

# 🔁 Haupt-Loop: alle Daten je Ticker abrufen & speichern
for ticker in tickers:
    print(f"\n🚀 Verarbeite: {ticker}")
    folder_path = os.path.join(base_path, ticker)
    os.makedirs(folder_path, exist_ok=True)

    try:
        # 📉 Kursdaten via yf.download
        hist = yf.download(ticker, group_by="ticker", auto_adjust=False)
        hist.reset_index(inplace=True)
        hist['Date'] = pd.to_datetime(hist['Date']).dt.strftime('%Y-%m-%d')
        save_json(hist, os.path.join(folder_path, "shareprices.json"))
    except Exception as e:
        print(f"❌ Kursdaten-Fehler bei {ticker}: {e}")

    try:
        # 🧠 Ticker-Objekt erstellen
        t = yf.Ticker(ticker)

        # 📄 Meta-Informationen
        save_json(t.info, os.path.join(folder_path, "info.json"))

        # 🗞 News
        save_json(t.news, os.path.join(folder_path, "news.json"))

        # 📆 Earnings
        save_json(t.earnings_dates, os.path.join(folder_path, "earning_dates.json"))

        # 🧾 Insider Transactions
        save_json(t.insider_transactions, os.path.join(folder_path, "insider_transactions.json"))

        # 🏢 Institutionelle Inhaber
        save_json(t.institutional_holders, os.path.join(folder_path, "institutional_holders.json"))

        # 📊 Fondsinhaber
        save_json(t.mutualfund_holders, os.path.join(folder_path, "mutualfund_holders.json"))

        # 📂 SEC-Filings
        save_json(t.sec_filings, os.path.join(folder_path, "sec_filings.json"))

        # 📈 Finanzdaten: Cashflow, Bilanz, GuV
        save_json(t.cashflow.transpose(), os.path.join(folder_path, "cashflow.json"))
        save_json(t.balance_sheet.transpose(), os.path.join(folder_path, "balance_sheet.json"))
        save_json(t.financials.transpose(), os.path.join(folder_path, "financials.json"))

    except Exception as e:
        print(f"❌ Fehler bei Ticker {ticker}: {e}")



🚀 Verarbeite: JPM


[*********************100%***********************]  1 of 1 completed


✅ Gespeichert: data/JPM/shareprices.json
✅ Gespeichert: data/JPM/info.json
✅ Gespeichert: data/JPM/news.json
✅ Gespeichert: data/JPM/earning_dates.json
✅ Gespeichert: data/JPM/insider_transactions.json
✅ Gespeichert: data/JPM/institutional_holders.json
✅ Gespeichert: data/JPM/mutualfund_holders.json
✅ Gespeichert: data/JPM/sec_filings.json
✅ Gespeichert: data/JPM/cashflow.json
✅ Gespeichert: data/JPM/balance_sheet.json
✅ Gespeichert: data/JPM/financials.json

🚀 Verarbeite: BAC


[*********************100%***********************]  1 of 1 completed


✅ Gespeichert: data/BAC/shareprices.json
✅ Gespeichert: data/BAC/info.json
✅ Gespeichert: data/BAC/news.json
✅ Gespeichert: data/BAC/earning_dates.json
✅ Gespeichert: data/BAC/insider_transactions.json
✅ Gespeichert: data/BAC/institutional_holders.json
✅ Gespeichert: data/BAC/mutualfund_holders.json
✅ Gespeichert: data/BAC/sec_filings.json
✅ Gespeichert: data/BAC/cashflow.json
✅ Gespeichert: data/BAC/balance_sheet.json
✅ Gespeichert: data/BAC/financials.json

🚀 Verarbeite: C


...

✅ Gespeichert: data/HSBA.L/sec_filings.json
✅ Gespeichert: data/HSBA.L/cashflow.json
✅ Gespeichert: data/HSBA.L/balance_sheet.json
✅ Gespeichert: data/HSBA.L/financials.json

🚀 Verarbeite: BNP.PA


[*********************100%***********************]  1 of 1 completed


✅ Gespeichert: data/BNP.PA/shareprices.json
✅ Gespeichert: data/BNP.PA/info.json
✅ Gespeichert: data/BNP.PA/news.json
✅ Gespeichert: data/BNP.PA/earning_dates.json
⚠️ Leere DataFrame: data/BNP.PA/insider_transactions.json
✅ Gespeichert: data/BNP.PA/institutional_holders.json
✅ Gespeichert: data/BNP.PA/mutualfund_holders.json


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BNP.PA?modules=secFilings&corsDomain=finance.yahoo.com&formatted=false&symbol=BNP.PA&crumb=4UXZu7h4SiE


✅ Gespeichert: data/BNP.PA/sec_filings.json
✅ Gespeichert: data/BNP.PA/cashflow.json
✅ Gespeichert: data/BNP.PA/balance_sheet.json
✅ Gespeichert: data/BNP.PA/financials.json
